### Your name:

Siqi Liu

### Collaborators:

Borrowed a snippet of code from [StackOverflow](https://stackoverflow.com/questions/33716395/predict-with-sklearn-knn-using-median-instead-of-mean)


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Markdown referece can be found here:
    http://nestacms.com/docs/creating-content/markdown-cheat-sheet

Q1. Why would it be a problem if our training set and test set are the same.

If our test set and training set are the same, we would get excellent model results but then we wouldn't know how our model would behave on an unfamiliar new set of data. Specifically, if the test/training sets are not representative of the true population, the model would not do well for a new set of data. Also, in the case of supervised learning, the model might try to fit every data in the dataset in order to maximize prediction accuracy, resulting in overfitting and poor prediction of new data.

Q2 [OPTIONAL]. Explain step by step the process to select k in the k-nearest neighbor algorithm (pseudocode) 

<pre> ENTER SOLUTION HERE </pre>

Q3. For the k-nearest regression. What happends when k = n. Where n is equal to the training size.

If k = n, then the model would make the same prediction for all new data points because all of the n training data points are the new data points' nearest neighbours.

Q4. Define a function that takes a 1-d numpy array, a parameter k, and a number p.
The function returns an estimate equal to the mean of the closest k points to the number p

In [2]:
import random

def k_neighbor(input_data, k, p):
    """Returns the k-neighbor estimate for p using data input_data.
    
    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    
    If you make assumptions please explain them in the comments. i.e. tie breaking strategy.
    """
    # If given empty array
    if len(input_data) == 0:
        print("input_data is empty!")
        return
    
    # If k > input_data length
    if k > len(input_data):
        k = len(input_data)
    
    # List of the absolute differences between p and all data points in input_data
    distances = abs(p - input_data)
    
    # Dictionary of (absolute difference, list of data points) pairs
    distances_dict = {}
    for i in range(len(distances)):
        if distances[i] in distances_dict:
            distances_dict[distances[i]].append(input_data[i])
        else:
            distances_dict[distances[i]] = [input_data[i]]
    if debug:
        print("********\np = {0}, k = {1}, distances = {2}".format(p, k, distances_dict))
    
    # Replace "distances" with a list of all unique distances
    distances = list(distances_dict.keys())
    distances.sort()
    
    # List of k nearest neighbours
    neighbours = []
    i = 0
    while len(neighbours) < k:
        dist = distances[i]
        n = len(distances_dict[dist])
        if debug:
            print("Adding {0}th neighbour from dist = {1}".format(i+1, dist))
        # Only one neighbour in the list, or more than one neighbour but adding all of the neighbours wouldn't exceed k
        if (n == 1) or (n > 1 and n <= k - len(neighbours)):
            neighbours += distances_dict[dist]
            i += n
            if debug:
                print("Neighbours: {0}".format(neighbours))
        # More than one neighbour, and adding them all would exceed k
        else:
            if debug:
                print("Randomly selecting {0} values from {1}".format(k-len(neighbours), distances_dict[dist]))
            # Randomly select the remaining neighbours
            neighbours += random.sample(population=distances_dict[dist], k=k-len(neighbours))
            assert len(neighbours) == k
            if debug:
                print("Neighbours: {0}".format(neighbours))
    
    # Return the mean
    return sum(neighbours) / k

In [3]:
#Evaluate
debug = False
data = np.array([1,3,4,5,7,8,11,12,13,15,19,24,25,29,40])
print(k_neighbor(input_data=data, k=3, p=5))
print(k_neighbor(input_data=data, k=2, p=15))
print(k_neighbor(input_data=data, k=3, p=25))
print(k_neighbor(input_data=data, k=1, p=55))
print(k_neighbor(input_data=data, k=3, p=55))
print(k_neighbor(input_data=data, k=10, p=55))

4.0
14.0
26.0
40.0
31.333333333333332
19.6


Observations:

1. While all data points in the array given are all integers, the returned values are not necessarily integers since we are taking the average. If we know for certain that only integers exist in the true population, then we should implement rounding at the end.
    
2. To handle ties between two numbers on either side of p, the implemention above randomly selects one of the numbers. In the example below, both 4 and 8 have distance of 2 from p, which is 6. After selecting the only 6 in the dataset, we need to select three more from two 4s and three 8s. We do this using "random.sample", and since there are more 8s than 4s, the 8s get selected more often. Without setting a seed for the sampling, we get a different result everytime we run it. Instead of random sampling, we could also:
    - Consistently select the number with the highest occurance, or
    - Select the weighted average of the two numbers. In the example below, that would be three times of (8 x 3 + 4 x 2) / 5 = 6.4, making the final ouput to be (6 + 6.4 x 3) / 4 ~= 6.3

In [4]:
debug = True
data_tie = np.array([6,8,8,8,4,4])
for i in range(10):
    print(k_neighbor(input_data=data_tie, k=4, p=6))

********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 8, 4, 8]
6.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 4, 8, 4]
5.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 4, 8, 8]
6.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 4, 8, 4]
5.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour fro

Q5. Similar to Q4 but for the n dimentional case. 

In [5]:
import math

def l1_norm(a, b):
    #sum(|x - y|)
    assert len(a) == len(b)
    return np.sum(np.abs(a - b))

def l2_norm(a, b):
    #sqrt(sum(|x - y|^2))
    assert len(a) == len(b)
    return np.sqrt(np.sum(np.square(a - b)))
    
def k_neighbor_nd(input_data, k, p, metric='l1', mode='mean'):
    """Returns the k-neighbor estimate for p using data input_data.

    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    metric -- l1 or l2. l1 norm or l2 norm https://en.wikipedia.org/wiki/Norm_(mathematics)
    mode -- estimator possible values = 'mean', 'median', 'max'
    
    Implement the l1 and l2 norms
    for mean, median and max, use np.mean, np.median and np.max
    """
    
    # If given empty array
    if len(input_data) == 0:
        print("input_data is empty!")
        return
    
    # If k > input_data length
    if k > len(input_data):
        k = len(input_data)
    
    # List of the distances between p and all data points in input_data
    distances = []
    if metric == 'l1':
        for i in input_data:
            distances.append(l1_norm(i, p))
    else:
        for i in input_data:
            distances.append(l2_norm(i, p))
    
    # Dictionary of (absolute difference, list of data points) pairs
    distances_dict = {}
    for i in range(len(distances)):
        if distances[i] in distances_dict:
            distances_dict[distances[i]].append(input_data[i])
        else:
            distances_dict[distances[i]] = [input_data[i]]
    if debug:
        print("********\np = {0}, k = {1}, distances = {2}".format(p, k, distances_dict))
    
    # Replace "distances" with a list of all unique distances
    distances = list(distances_dict.keys())
    distances.sort()
    
    # List of k nearest neighbours
    neighbours = []
    i = 0
    while len(neighbours) < k:
        dist = distances[i]
        n = len(distances_dict[dist])
        if debug:
            print("Adding {0}th neighbour from dist = {1}".format(i+1, dist))
        # Only one neighbour in the list, or more than one neighbour but adding all of the neighbours wouldn't exceed k
        if (n == 1) or (n > 1 and n <= k - len(neighbours)):
            neighbours += distances_dict[dist]
            i += n
            if debug:
                print("Neighbours: {0}".format(neighbours))
        # More than one neighbour, and adding them all would exceed k
        else:
            if debug:
                print("Randomly selecting {0} values from {1}".format(k-len(neighbours), distances_dict[dist]))
            # Randomly select the remaining neighbours
            neighbours += random.sample(population=distances_dict[dist], k=k-len(neighbours))
            assert len(neighbours) == k
            if debug:
                print("Neighbours: {0}".format(neighbours))
    
    # Return the value
    if mode == 'mean':
        result = np.mean(neighbours, axis=0)
    elif mode == 'median':
        result = np.median(neighbours, axis=0)
    elif mode == 'max':
        result = np.max(neighbours, axis=0)
    
    # Calculate the result's distance to p
    if debug:
        if metric == 'l1':
            print("Result has a distance of {0} with p".format(l1_norm(result, p)))
        else:
            print("Result has a distance of {0} with p".format(l2_norm(result, p)))
    
    return result

In [6]:
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])

In [7]:
data_4d

array([[4, 1, 2, 1],
       [1, 4, 2, 0],
       [3, 3, 1, 1],
       [4, 0, 0, 0],
       [1, 2, 0, 0],
       [3, 4, 2, 3],
       [2, 4, 4, 2],
       [2, 1, 4, 1],
       [3, 3, 2, 4],
       [4, 3, 0, 4],
       [2, 2, 4, 0],
       [4, 3, 0, 2],
       [4, 3, 0, 2],
       [0, 3, 4, 2]])

In [8]:
#Evaluate
debug = False
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l2', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='median'))

# Other test cases
debug = True
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='max'))

[2.         2.33333333 4.         1.        ]
[4. 3. 0. 2.]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]
[1.66666667 2.33333333 3.33333333 2.33333333]
[3.5 3.  0.5 1.5]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]
********
p = [2, 3, 3, 4], k = 3, distances = {8: [array([4, 1, 2, 1])], 7: [array([1, 4, 2, 0]), array([4, 3, 0, 2]), array([4, 3, 0, 2])], 6: [array([3, 3, 1, 1]), array([2, 1, 4, 1]), array([2, 2, 4, 0])], 12: [array([4, 0, 0, 0])], 9: [array([1, 2, 0, 0])], 4: [array([3, 4, 2, 3]), array([2, 4, 4, 2])], 2: [array([3, 3, 2, 4])], 5: [array([4, 3, 0, 4]), array([0, 3, 4, 2])]}
Adding 1th neighbour from dist = 2
Neighbours: [array([3, 3, 2, 4])]
Adding 2th neighbour from dist = 4
Neighbours: [array([3, 3, 2, 4]), array([3, 4, 2, 3]), array([2, 4, 4, 2])]
Result has a distance of 2.6666666666666665 with p
[2.66666667 3.66666667 2.66666667 3.        ]
********
p = [2, 3, 3, 4], k = 3, distances = {8: [array([4, 1, 2, 1])], 7: [array([1, 4, 2, 0]), array([4, 3, 0, 2]), array([4, 3, 0, 2])], 

Observations:

1. Similar to Q4, if we know that only points with integer coordiates can exist in the population, then we should consider rounding at the end
2. Using the same training dataset, k and p, but different metrics (i.e. L1 or L2), the selected neighbours and thus the results can be very different. This suggests that choosing a distance function that is suitable for the model is very important
3. Even under the same metrics, different modes (i.e. mean/median/max) can also generate vastly different results. This is because 
4. With 4d array, it's much more difficult for two to have the same distance to p, hence we see less ties

Q6[Optional]. Read the documentation on KNeighborsRegressor

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
    
Explore the source code:
    https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/regression.py
        
How different it is from your implementation? How well can you follow the code? Did you learn something new?

In [9]:
import sklearn
from sklearn import linear_model

In [10]:
#Q4 redone

print("***MINE***")
debug = False
data = np.array([1,3,4,5,7,8,11,12,13,15,19,24,25,29,40])
print(k_neighbor(input_data=data, k=3, p=5))
print(k_neighbor(input_data=data, k=2, p=15))
print(k_neighbor(input_data=data, k=3, p=25))
print(k_neighbor(input_data=data, k=1, p=55))
print(k_neighbor(input_data=data, k=3, p=55))
print(k_neighbor(input_data=data, k=10, p=55))
    
print("***SKLEARN***")
data = data.reshape(-1, 1)
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3).fit(X=data, y=data).predict([[5]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=2).fit(X=data, y=data).predict([[15]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3).fit(X=data, y=data).predict([[25]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=1).fit(X=data, y=data).predict([[55]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3).fit(X=data, y=data).predict([[55]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=10).fit(X=data, y=data).predict([[55]])))

***MINE***
4.0
14.0
26.0
40.0
31.333333333333332
19.6
***SKLEARN***
4.0
14.0
26.0
40.0
31.333333333333332
19.6


In [11]:
#Q4 tie cases redone

print("***MINE***")
debug = True
data_tie = np.array([6,8,8,8,4,4])
for i in range(5):
    print(k_neighbor(input_data=data_tie, k=4, p=6))
    
print("***SKLEARN***")
data_tie = data_tie.reshape(-1, 1)
for i in range(5):
    print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=4).fit(X=data_tie, y=data_tie).predict([[6]]))) # 6 + 8 + 8 + 4

print("***SKLEARN NEIGHBOURS***")
sklearn_neighbours = np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=4).fit(X=data_tie, y=data_tie).kneighbors([[6]], return_distance=False))
for i in sklearn_neighbours:
    print(data_tie[i])

***MINE***
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 8, 4, 4]
5.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 8, 4, 4]
5.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 8, 8, 4]
6.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th neighbour from dist = 0
Neighbours: [6]
Adding 2th neighbour from dist = 2
Randomly selecting 3 values from [8, 8, 8, 4, 4]
Neighbours: [6, 4, 8, 8]
6.5
********
p = 6, k = 4, distances = {0: [6], 2: [8, 8, 8, 4, 4]}
Adding 1th ne

In [12]:
#Q5 redone, but only with mean, since native sklearn KNN doesn't have option for median & max
print("***MINE***")
debug = False
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='mean'))
    
print("***SKLEARN***")
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3, p=1).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=2, p=1).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=1, p=1).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3, p=2).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=2, p=2).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=1, p=2).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))

***MINE***
[2.         2.33333333 4.         1.        ]
[4. 3. 0. 2.]
[3. 3. 2. 4.]
[1.33333333 2.66666667 4.         1.66666667]
[3.5 3.  0.5 1.5]
[3. 3. 2. 4.]
***SKLEARN***
[2.         2.33333333 4.         1.        ]
[4. 3. 0. 2.]
[3. 3. 2. 4.]
[1.33333333 2.66666667 4.         1.66666667]
[3.5 3.  0.5 1.5]
[3. 3. 2. 4.]


In [13]:
#Q5 tie case
print("***MINE***")
debug = True
for i in range(5):
    print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
    
print("***SKLEARN***")
for i in range(5):
    print(np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3, p=2).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])
                    ))
print("***SKLEARN NEIGHBOURS***")
sklearn_neighbours = np.squeeze(sklearn.neighbors.KNeighborsRegressor(n_neighbors=3, p=2).fit(X=data_4d, y=data_4d).kneighbors([[2, 1, 4, 3]], return_distance=False))
for i in sklearn_neighbours:
    print(data_4d[i])

***MINE***
********
p = [2, 1, 4, 3], k = 3, distances = {3.4641016151377544: [array([4, 1, 2, 1])], 4.795831523312719: [array([1, 4, 2, 0])], 4.242640687119285: [array([3, 3, 1, 1])], 5.477225575051661: [array([4, 0, 0, 0])], 5.196152422706632: [array([1, 2, 0, 0])], 3.7416573867739413: [array([3, 4, 2, 3])], 3.1622776601683795: [array([2, 4, 4, 2]), array([3, 3, 2, 4]), array([2, 2, 4, 0])], 2.0: [array([2, 1, 4, 1])], 5.0: [array([4, 3, 0, 4]), array([4, 3, 0, 2]), array([4, 3, 0, 2])], 3.0: [array([0, 3, 4, 2])]}
Adding 1th neighbour from dist = 2.0
Neighbours: [array([2, 1, 4, 1])]
Adding 2th neighbour from dist = 3.0
Neighbours: [array([2, 1, 4, 1]), array([0, 3, 4, 2])]
Adding 3th neighbour from dist = 3.1622776601683795
Randomly selecting 1 values from [array([2, 4, 4, 2]), array([3, 3, 2, 4]), array([2, 2, 4, 0])]
Neighbours: [array([2, 1, 4, 1]), array([0, 3, 4, 2]), array([2, 2, 4, 0])]
Result has a distance of 2.3333333333333335 with p
[1.33333333 2.         4.         1.  

In [14]:
# Copied from https://stackoverflow.com/questions/33716395/predict-with-sklearn-knn-using-median-instead-of-mean
from sklearn.neighbors.regression import KNeighborsRegressor, check_array, _get_weights

class MedianKNNRegressor(KNeighborsRegressor):
    def predict(self, X):
        X = check_array(X, accept_sparse='csr')

        neigh_dist, neigh_ind = self.kneighbors(X)

        weights = _get_weights(neigh_dist, self.weights)

        _y = self._y
        if _y.ndim == 1:
            _y = _y.reshape((-1, 1))

        ######## Begin modification
        if weights is None:
            y_pred = np.median(_y[neigh_ind], axis=1)
        else:
            # y_pred = weighted_median(_y[neigh_ind], weights, axis=1)
            raise NotImplementedError("weighted median")
        ######### End modification

        if self._y.ndim == 1:
            y_pred = y_pred.ravel()

        return y_pred

In [15]:
# Max
from sklearn.neighbors.regression import KNeighborsRegressor, check_array, _get_weights

class MaxKNNRegressor(KNeighborsRegressor):
    def predict(self, X):
        X = check_array(X, accept_sparse='csr')

        neigh_dist, neigh_ind = self.kneighbors(X)

        weights = _get_weights(neigh_dist, self.weights)

        _y = self._y
        if _y.ndim == 1:
            _y = _y.reshape((-1, 1))

        ######## Begin modification
        if weights is None:
            y_pred = np.max(_y[neigh_ind], axis=1)
        else:
            # y_pred = weighted_median(_y[neigh_ind], weights, axis=1)
            raise NotImplementedError("weighted max")
        ######### End modification

        if self._y.ndim == 1:
            y_pred = y_pred.ravel()

        return y_pred

In [16]:
#Q5 redone, with median
print("***MINE***")
debug = False
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='median'))
    
print("***SKLEARN***")
print(np.squeeze(MedianKNNRegressor(n_neighbors=3, p=1).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(MedianKNNRegressor(n_neighbors=2, p=1).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(MedianKNNRegressor(n_neighbors=1, p=1).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))
print(np.squeeze(MedianKNNRegressor(n_neighbors=3, p=2).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(MedianKNNRegressor(n_neighbors=2, p=2).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(MedianKNNRegressor(n_neighbors=1, p=2).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))

***MINE***
[2. 2. 4. 1.]
[4. 3. 0. 2.]
[3. 3. 2. 4.]
[2. 2. 4. 1.]
[3.5 3.  0.5 1.5]
[3. 3. 2. 4.]
***SKLEARN***
[2. 2. 4. 1.]
[4. 3. 0. 2.]
[3. 3. 2. 4.]
[2. 3. 4. 2.]
[3.5 3.  0.5 1.5]
[3. 3. 2. 4.]


In [17]:
#Q5 redone, with max
print("***MINE***")
debug = False
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='max'))
    
print("***SKLEARN***")
print(np.squeeze(MaxKNNRegressor(n_neighbors=3, p=1).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(MaxKNNRegressor(n_neighbors=2, p=1).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(MaxKNNRegressor(n_neighbors=1, p=1).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))
print(np.squeeze(MaxKNNRegressor(n_neighbors=3, p=2).fit(X=data_4d, y=data_4d).predict([[2, 1, 4, 3]])))
print(np.squeeze(MaxKNNRegressor(n_neighbors=2, p=2).fit(X=data_4d, y=data_4d).predict([[4, 4, 0, 0]])))
print(np.squeeze(MaxKNNRegressor(n_neighbors=1, p=2).fit(X=data_4d, y=data_4d).predict([[2, 3, 3, 4]])))

***MINE***
[2 4 4 2]
[4 3 0 2]
[3 3 2 4]
[2 4 4 2]
[4 3 1 2]
[3 3 2 4]
***SKLEARN***
[2 4 4 2]
[4 3 0 2]
[3 3 2 4]
[2 4 4 2]
[4 3 1 2]
[3 3 2 4]


SkLearn's KNN seems to not randomly select neighbours when there is a tie. Looking at the [source code](https://github.com/scikit-learn/scikit-learn/blob/952ef6637a13ba01055bf0ed9d29d525f0fee5bc/sklearn/neighbors/base.py#L332) for neighbor selection on SkLearn's GitHub page, it seems like they've done the following:

If the method is "brute force" (which is in this case), the `kneighbors` function gets called, and the `neigh_ind` indicator array is calculated using `pairwise` distance calculation, then gets [sorted](https://github.com/scikit-learn/scikit-learn/blob/df9f90cfa8795b6d85056f70177fb783d6ecafda/sklearn/neighbors/base.py#L356). And then, it [always selects the first ones from the duplicate group](https://github.com/scikit-learn/scikit-learn/blob/df9f90cfa8795b6d85056f70177fb783d6ecafda/sklearn/neighbors/base.py#L392) instead of taking a random sample. This ensures that the resulting neighbours are always consistent